In [13]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import boxcox
pd.options.display.max_rows = 50
pd.options.display.max_columns = 999
import warnings
warnings.filterwarnings('ignore')

<b>  working with new csv file name Data_Marketing_Customer_Analysis_Round2.csv that has Response and Sales Channel

In [ ]:
def load_original_data():
    return pd.read_csv("Data/Data_Marketing_Customer_Analysis_Round2.csv" )

In [ ]:
def clean_data(df):
    df.columns=[e.lower().replace(' ', '_') for e in df.columns]
    df=df.drop(columns=['unnamed:_0', 'vehicle_type',"customer"])
    df=df.dropna()
    df['effective_to_date']=pd.to_datetime(df['effective_to_date'], errors='coerce')
    df["month"]=list(map(lambda date:date.strftime(format="%B"),df["effective_to_date"]))
    # we drop effective_to_date column
    df=df.drop(columns=['effective_to_date'])
    ## removing duplicates
    df=df.drop_duplicates()
    return df

In [ ]:
def transform(df):
    df=df[df['income']>0]
    df["total_claim_amount"]=round(df["total_claim_amount"],0)
    df["customer_lifetime_value"]=round(df["customer_lifetime_value"],0)
    df.vehicle_class[customers_df.vehicle_class.isin(["Luxury Car","Luxury SUV"])] = "Luxury Car"
    df.vehicle_class[customers_df.vehicle_class.isin(["Four-Door Car","SUV"])] = "Four-Door Car"
    df.policy[df.policy.isin(["Personal L3","Personal L2","Personal L1"]) ] = "Personal"
    df.policy[df.policy.isin(["Corporate L3","Corporate L2","Corporate L1"]) ] = "Corporate"
    df.policy[df.policy.isin(["Special L3","Special L2","Special L1"]) ] = "Special"
    df=df.drop(columns=["month","education"])
    df.number_of_policies[df.number_of_policies>2]=3
    return df

In [ ]:
customers_df=load_original_data()
customers_df=clean_data(customers_df)
customers_df=transform(customers_df)

In [ ]:
customers_df.shape

In [ ]:
customers_df.columns

# Plotting

EDA Catgorical Columns 

In [3]:
for c in customers_df.select_dtypes(object):
    plt.figure(figsize=(10,5))
    plt.bar(customers_df[c].unique(), customers_df[c].value_counts())
    plt.title(c)
    plt.show();

NameError: name 'customers_df' is not defined

### Removing outliers

<b> customer life time value

In [ ]:
sns.boxplot(customers_df.customer_lifetime_value)

In [ ]:
Q1 = customers_df.customer_lifetime_value.quantile(0.25)
Q3 = customers_df.customer_lifetime_value.quantile(0.75)
IQR = Q3 - Q1    #IQR is interquartile range.

In [ ]:
filter = (customers_df["customer_lifetime_value"] >= Q1 - 1.5 * IQR) & (customers_df["customer_lifetime_value"] <= Q3 + 1.5 *IQR)
customers_df=customers_df.loc[filter] 

In [ ]:
customers_df.customer_lifetime_value.hist()

<b> Monthly Premium Auto

In [ ]:
sns.boxplot(customers_df.monthly_premium_auto)

In [ ]:
Q1 = customers_df.monthly_premium_auto.quantile(0.25)
Q3 = customers_df.monthly_premium_auto.quantile(0.75)
IQR = Q3 - Q1    #IQR is interquartile range.

In [ ]:
IQR

In [ ]:
filter = (customers_df["monthly_premium_auto"] >= Q1 - 1.5 * IQR) & (customers_df["monthly_premium_auto"] <= Q3 + 1.5 *IQR)
customers_df=customers_df.loc[filter]

In [ ]:
customers_df

### Show a plot of the response rate by the sales channel.

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot('response', hue='sales_channel', data=customers_df)
plt.ylabel('Response by Sales Channel')
plt.show()


### Show a plot of the response rate by the total claim amount.


In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(y='total_claim_amount' , x='response', data=customers_df)
plt.ylabel('Response by Total Claim Amount')
plt.show()


### Show a plot of the response rate by income.


In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(y='income' , x='response', data=customers_df)
plt.ylabel('Response by Income')
plt.show()

# Data Transformation Lab

#### Check normality visually.

In [ ]:
sns.pairplot(customers_df)
plt.show()

In [4]:
sns.distplot((customers_df['customer_lifetime_value']), bins=20)

NameError: name 'sns' is not defined

In [ ]:
customer_lifetime_value_transformed=boxcox(customers_df['customer_lifetime_value'],lmbda=-0.04)

In [ ]:
sns.histplot(customer_lifetime_value_transformed)

In [ ]:
sns.distplot((customers_df['monthly_premium_auto']), bins=20)

In [ ]:
pt = PowerTransformer()
monthly_premium_auto_transformed=pt.fit_transform(customers_df['monthly_premium_auto'].to_numpy().reshape(-1,1))
sns.displot(monthly_premium_auto_transformed)

In [ ]:
monthly_premium_auto_transformed,lam=boxcox(customers_df['monthly_premium_auto'])
sns.histplot(monthly_premium_auto_transformed)

In [ ]:
sns.regplot(x='monthly_premium_auto',y='total_claim_amount', data=customers_df, scatter_kws={"color": "olive"}, line_kws={"color": "black"})

#### Checking Multicorrelinilaty

In [5]:
numericals_features=customers_df._get_numeric_data()
correlations = numericals_features.corr()
correlations

NameError: name 'customers_df' is not defined

In [ ]:
cmap = sns.cm.rocket_r  ## for reversing the colors
mask=np.zeros_like(numericals_features.corr())
mask[np.triu_indices_from(mask)]=True
fig, ax=plt.subplots(figsize=(10, 8))
ax=sns.heatmap(numericals_features.corr(), mask=mask, annot=True,cmap=cmap)
plt.show()

X-Y split  
splitting the target variable from the dependent variables

In [ ]:
X=customers_df.drop('total_claim_amount', axis=1)
y=customers_df.total_claim_amount

In [6]:
X

NameError: name 'X' is not defined

Get the numeric data.

In [ ]:
numericals_features =X._get_numeric_data()
# numericals =data.select_dtypes(np.number)
numericals_features.head()

In [ ]:
numericals_features.describe()

Get Categorical data

In [ ]:
categoricals_features = X.select_dtypes(object)
categoricals_features.head()

### Categorical Variables Transformation

One Hot Encoding

In [ ]:
# get dummies pandas
categoricals_features=pd.get_dummies(categoricals_features, drop_first=True)
##pd.DataFrame(OneHotEncoder(drop='first').fit_transform(categoricals_features).toarray(),
## columns=OneHotEncoder(drop='first').fit(categoricals_features).get_feature_names(input_features=categoricals_features.columns)).head()

In [ ]:
categoricals_features

In [ ]:
all_features=pd.concat([numericals_features, categoricals_features], axis=1) # concat numerical and categorical transformations
all_features.head()

### Train Test Split

In [ ]:
all_features=significant_features

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(all_features, y, test_size=.20)

In [ ]:
X_train

#### Standardize

In [ ]:
std_scaler=StandardScaler().fit(X_train) 

X_train_scaled=std_scaler.transform(X_train)

In [ ]:
X_test_scaled=std_scaler.transform(X_test)

In [7]:
X_train_scaled[0]

NameError: name 'X_train_scaled' is not defined

### Modeling using Statsmodels

<b>without scaling

In [ ]:
X_train_const= sm.add_constant(X_train) # adding a constant

model = sm.OLS(y_train, X_train_const).fit()
predictions_train = model.predict(X_train_const) 

X_test_const = sm.add_constant(X_test) # adding a constant
predictions_test = model.predict(X_test_const) 
print_model = model.summary()
print(print_model)

### checking the significant variables

In [ ]:
model.params[list(np.where(model.pvalues < 0.05)[0])].iloc[1:].index.tolist()

In [ ]:
significant_features=all_features[model.params[list(np.where(model.pvalues < 0.05)[0])].iloc[1:].index.tolist()]

<b> with scaling

In [ ]:
X_train.columns

In [ ]:
X_train_const_scaled = sm.add_constant(X_train_scaled) # adding a constant

model = sm.OLS(y_train, X_train_const_scaled).fit()
predictions_train = model.predict(X_train_const_scaled) 

X_test_const_scaled = sm.add_constant(X_test_scaled) # adding a constant
predictions_test = model.predict(X_test_const_scaled) 
print_model = model.summary()
print(print_model)

### Apply Linear Regression

In [ ]:
model=LinearRegression()    # model
model.fit(X_train_scaled, y_train)   # model train

y
y_pred=model.predict(X_test_scaled)   # model prediction


y_pred_train=model.predict(X_train_scaled)

In [ ]:
# Make an scatter plot y_pred vs y
# What kind of plot you will get if all the all the predictions are ok?
# A stright line

fig, ax = plt.subplots(2,3,figsize=(14,12))
ax[0,0].plot(y_pred, y_test, 'o')
ax[0,0].set_xlabel("y_test")
ax[0,0].set_ylabel("y_pred")
ax[0,0].set_title("Test Set -Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscdasticity
# It resembles a normal distribution?
ax[0,1].hist(y_test - y_pred)
ax[0,1].set_xlabel("Test y-y_pred")
ax[0,1].set_title("Test Set Residual histogram")

ax[0,2].plot(y_pred,y_test - y_pred,"o")
ax[0,2].set_xlabel("predited")
ax[0,2].set_ylabel("residuals")
ax[0,2].set_title("Residuals by Predicted -- Test set")
ax[0,2].plot(y_pred,np.zeros(len(y_pred)),linestyle='dashed')


ax[1,0].plot(y_pred_train, y_train, 'o')
ax[1,0].set_xlabel("y_train")
ax[1,0].set_ylabel("y_pred_train")
ax[1,0].set_title("Train set Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscdasticity
# It resembles a normal distribution?
ax[1,1].hist(y_train - y_pred_train)
ax[1,1].set_xlabel("Train y-y_pred")
ax[1,1].set_title("Train Residual histogram")

ax[1,2].plot(y_pred_train,y_train - y_pred_train,"o")
ax[1,2].set_xlabel("predited")
ax[1,2].set_ylabel("residuals")
ax[1,2].set_title("Residuals by Predicted -- Train set")
ax[1,2].plot(y_pred_train,np.zeros(len(y_pred_train)),linestyle='dashed')

#### Model Validation

MSE.

In [ ]:
train_mse=mse(y_train,y_pred_train)
test_mse=mse(y_test,y_pred)

print ('train MSE: {} -- test MSE: {}'.format(train_mse, test_mse))

RMSE.

In [ ]:
print ('train RMSE: {} -- test RMSE: {}'.format(train_mse**.5, test_mse**.5))

MAE.

In [ ]:
train_mae=mae(y_train,y_pred_train)
test_mae=mae(y_test,y_pred)

print ('train MAE: {} -- test MAE: {}'.format(train_mse, test_mse))

R2.

In [ ]:
#R2= model.score(X_test_scaled, y_test)
R2_train=r2_score(y_train,y_pred_train)
R2_test=r2_score(y_test,y_pred)

print (R2_train)
print(R2_test)

In [ ]:
print ('train R2: {} -- test R2: {}'.format(model.score(X_train_scaled, y_train),
                                            model.score(X_test_scaled, y_test)))

adjusted rsquare

In [ ]:
Adj_R2_train= 1 - (1-R2_train)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
Adj_R2_train

In [ ]:
Adj_R2_test= 1 - (1-R2_test)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
Adj_R2_test

#### Feature Importance

In [ ]:
features_importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': abs(model.coef_)
})
features_importances = features_importances.sort_values(by='Importance', ascending=False)

In [ ]:
## decide on each variable importance on your model using adjusted R_squared 

In [ ]:
features_importances

In [ ]:
plt.bar(x=features_importances['Attribute'], height=features_importances['Importance'], color='Orange')
plt.title('Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()